# Entanglement Boosting: Step-by-step Walkthrough

This notebook introduces `stim` basics, then walks through the key pieces of this repo, and finally runs a tiny entanglement boosting simulation.

## 0. Setup and imports
If you haven't installed dependencies yet, run: `pip install -r requirements.txt`.

In [ ]:
import sys
import stim
import pymatching

# Make sure we can import local modules
if 'src' not in sys.path:
    sys.path.append('src')

import entanglement_boosting as eb
import util
import surface_code

print('stim', stim.__version__)
print('pymatching', pymatching.__version__)

## 1. Stim basics: build a tiny circuit
We start with a single qubit, apply H, measure in Z, and inspect the circuit.

In [ ]:
c = stim.Circuit()
c.append('H', 0)
c.append('M', 0)
print(c)

## 2. Stim basics: sampling
Use `compile_sampler()` to produce measurement samples.

In [ ]:
sampler = c.compile_sampler()
samples = sampler.sample(shots=10)
samples

## 3. Repo utility layer: Circuit wrapper
This project wraps stim.Circuit to enforce nearest-neighbor CX, one-gate-per-tick, and to inject noise.

In [ ]:
# Create a tiny mapping and a noiseless circuit
mapping = util.QubitMapping(width=3, height=3)
noise = util.NO_ERROR_CONF
cwrap = util.Circuit(mapping, noise)

# Place a few operations using the wrapper
cwrap.place_reset_x((0, 0))
cwrap.place_reset_z((2, 0))
cwrap.place_cx((0, 0), (1, 1))
cwrap.place_measurement_z((2, 0))
cwrap.place_tick()

print(cwrap.circuit)

## 4. Surface-code patch layout
The core logic builds two patches and performs syndrome measurement rounds.
We can instantiate a small patch just to inspect its syndrome measurement positions.

In [ ]:
mapping = util.QubitMapping(width=9, height=5)
cwrap = util.Circuit(mapping, util.NO_ERROR_CONF)
patch = eb.SurfaceCodePatch(cwrap, offset=(1, 1), bell_distance=2, distance=3)

len(patch.syndrome_measurements), list(patch.syndrome_measurements.keys())[:5]

## 5. Build the entanglement boosting circuit (small size)
Use very small distances to keep it fast and readable.

In [ ]:
noise_conf = util.NoiseConfiguration(
    single_qubit_gate_error_probability=0.0,
    two_qubit_gate_error_probability=0.0,
    reset_error_probability=0.0,
    measurement_error_probability=0.0,
    idle_error_probability=0.0,
)

circuits = eb.perform_distillation(
    bell_distance=2,
    surface_distance=3,
    noise_conf=noise_conf,
    bell_error_probability=0.0,
    post_selection=False,
)

print(circuits.circuit.circuit)

## 6. Simulate a few shots
Run a tiny number of shots to see how results are collected.

In [ ]:
results = eb.perform_simulation(circuits, num_shots=20, seed=0)
len(results), results.num_discarded_samples

## 7. Inspect complementary gap buckets
The code buckets outcomes by the complementary gap used for post-selection.

In [ ]:
# Show a few non-empty buckets
non_empty = [(i, b) for i, b in enumerate(results.buckets) if len(b) > 0]
non_empty[:5]

## 8. Next steps
- Increase distances and introduce noise
- Enable post-selection (`post_selection=True`)
- Use `perform_parallel_simulation` for larger runs